<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan_extras; plan_extras("numpy", "struct")

# structured arrays

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

* jusqu'ici on a vu des tableaux *homogènes* 
  * tous les éléments ont le même type
* on peut aussi se définir des types structurés
  * comme un 'struct' en c - ou encore un 'record'
* demande un peu plus d'efforts au programmeur

## exemple

In [ ]:
classe = np.array(
  # les données sont une liste d'éléments homogènes
  [ 
   # mais cette fois chaque élément (ligne) est un composite
   # que l'on peut décrire dans un tuple
    ( 'Jean', 'Dupont', 32),
   # tous les tuples doivent avoir la même structure
    ( 'Daniel', 'Durand', 18),  ( 'Joseph', 'Delapierre', 54),
    ( 'Paul', 'Girard', 20)])

In [ ]:
print(classe)

### `dtype`

In [ ]:
classe.dtype

comme pour les tableaux homogènes:

* comme je n'ai pas précisé de type
* c'est numpy qui en choisi un pour moi
* ici le plus petit dénominateur commun c'est le type string 
* de taille 10 d'ailleurs

* j'ai encore tableau homogène
* et d'ailleurs je peux toujours perdre de la précision

In [ ]:
classe[0, 0]

In [ ]:
classe[0, 0] = 'Charles-Henri'
classe

## spécifier `dtype`

* c'est important d'élaborer un type
* d'autant que toutes les colonnes ne sont pas identiques

In [ ]:
classe2 = np.array([ ( 'Jean', 'Dupont', 32),
                     ( 'Daniel', 'Durand', 18),
                     ( 'Joseph', 'Delapierre', 54),
                     ( 'Paul', 'Girard', 20)],
    dtype = [('prenom', '|S12'), ('nom', '|S15'), ('age', np.int)])
print(classe2)

## impact sur `shape` et `reshape`

* dans le cas de cette nouvelle définition
* `shape` retourne .. une seule dimension !

In [ ]:
# on peut faire reshape (même si ça
# ne pas beaucoup de sens de toutes façons)
print(classe.reshape(3, 4))
# car la dimension est habituelle
print("shape", classe.shape)

In [ ]:
# on ne peut pas faire reshape
try: print(classe2.reshape(3, 4))
except Exception as e: print("OOPS", e)
# la dimension n'est pas ce qu'on
# pourrait attendre
print("shape", classe2.shape)

### dimensions supérieures

* on peut sans souci créer des dimensions supérieures
* attention toutefois
  * comme c'est un tableau de dimension 1
  * c'est considéré comme **une ligne** 
  * dans le contexte d'un tableur
  * on préfèrerait peut-être que ce soit présenté en colonnes

In [ ]:
# on peut créer un tableau 2 x 4
# par exemple en superposant la même ligne 2 fois
classe2x2 = np.vstack( (classe2, classe2))
print(classe2x2)
print("shape = ",classe2x2.shape)

## comment définir `dtype`

* il existe plusieurs méthodes pour définir un `dtype` pour un 'structured array',
* sachant que par ailleurs un élément de la structure peut être à son tour un tableau

1. String
1. Tuple
1. List
1. Dict (2 formes)

* voire aussi comme [un objet `dtype`](https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html)

### `dtype` défini comme un string

* séparé par des virgules
* des codes comme e.g. `i8` (entier sur 64 bits) ou `a<12>` (string de taille 12)
* ou encore `float64` (cette fois en bits !)
* inconvénient: les champs ne sont pas nommés

In [ ]:
structs = np.ones(3, dtype='3int8, float32, (2,3)float64')
print(structs)

In [ ]:
# on peut accéder à tous les morceaux par indices
structs[1][2][1][1] *= 20
print(structs)

### `dtype` défini comme un tuple

permet de définir quelque chose qui ressemble à un `union`:

In [ ]:
powers = 1 + 8 * np.arange(4)
print(powers)

In [ ]:
unions = np.array([2**power for power in powers], 
                   dtype=('i4',[('r','u1'), ('g','u1'), ('b','u1'), ('a','u1')]))
print(unions)

In [ ]:
unions['r']

In [ ]:
unions['g']

In [ ]:
unions['b']

### `dtype` défini comme une liste

* déjà vu l'exemple de `classe2`
* doit être une liste de 2-tuples `nom` , `type`
* les noms peuvent servir à indexer (c'est tout l'intérêt)

In [ ]:
structs = np.ones(3, dtype=[('x','f4'),('y',np.float32),('value','f4',(2,2))])
structs

In [ ]:
structs[0]['x'] *= 10
structs[1]['value'][1][1] *= 100
# on peut accéder au second flottant par indice aussi
structs[2][1] *= 1000
structs

### `dtype` défini comme un dict(1)

* le dictionnaire a deux clés prédéfinies
  * `names` et `formats`
  * listes de même longueurs

In [ ]:
structs = np.ones(3, dtype={'names':['col1', 'col2'], 'formats':['i4','(2,3)f4']})
print(structs)

In [ ]:
structs[1]['col2'] *= 30
print(structs)

### `dtype` défini comme un dict(2)

* sinon, on s'attend à trouver dans `dtype`:
  * les clés sont les noms des colonnes
  * la valeur associée doit être un 2- ou 3-tuple
  * de la forme `(type, offset[, nom_colonne])`

In [ ]:
structs = np.ones(3, dtype={'col1':('i1',0,'title 1'), 'col2':('f4',1,'title 2')})
print(structs)

In [ ]:
structs[1]['col2'] *= np.pi
print(structs)

## exercice

* [Voir la documentation complète ici](https://docs.scipy.org/doc/numpy-1.10.1/user/basics.rec.html#defining-structured-arrays)
* sur la définition de types structurés

* On veut modéliser le groupe des 8 éléments du groupe D4 (les rotations et symétries d'ordre 4)
* sous la forme d'un tableau de 8 valeurs
* chacune ayant 
  * un nom (sur deux caractères)
  * une matrice carrée 2x2

[extrait de https://en.wikipedia.org/wiki/Dihedral_group](https://en.wikipedia.org/wiki/Dihedral_group)

$
{\displaystyle {\begin{matrix}\mathrm {r} _{0}=\left({\begin{smallmatrix}1&0\\[0.2em]0&1\end{smallmatrix}}\right),&\mathrm {r} _{1}=\left({\begin{smallmatrix}0&-1\\[0.2em]1&0\end{smallmatrix}}\right),&\mathrm {r} _{2}=\left({\begin{smallmatrix}-1&0\\[0.2em]0&-1\end{smallmatrix}}\right),&\mathrm {r} _{3}=\left({\begin{smallmatrix}0&1\\[0.2em]-1&0\end{smallmatrix}}\right),\\[1em]\mathrm {s} _{0}=\left({\begin{smallmatrix}1&0\\[0.2em]0&-1\end{smallmatrix}}\right),&\mathrm {s} _{1}=\left({\begin{smallmatrix}0&1\\[0.2em]1&0\end{smallmatrix}}\right),&\mathrm {s} _{2}=\left({\begin{smallmatrix}-1&0\\[0.2em]0&1\end{smallmatrix}}\right),&\mathrm {s} _{3}=\left({\begin{smallmatrix}0&-1\\[0.2em]-1&0\end{smallmatrix}}\right).\end{matrix}}}
$

In [ ]:
# les données sous forme de python 'standard'
d4_data = [ 
    ('r0', [[1, 0], [0, 1]]),      ('r1', [[0, -1], [1, 0]]),
    ('r2', [[-1, 0], [0, -1]]),    ('r3', [[0, 1], [-1, 0]]),
    ('s0', [[1, 0], [0, -1]]),     ('s1', [[0, 1], [1, 0]]),
    ('s2', [[-1, 0], [0, 1]]),     ('s3', [[0, -1], [-1, 0]]),
  ]

vous devez donc écrire quelque chose comme ceci

````
d4 = np.array( 
  d4_data,
  dtype = <votre code>
)
d4
```

### string

In [ ]:
# on ne pourra pas accéder aux champs par nom
D4 = np.array( d4_data,
  dtype = 'S2, (2,2)int8'
)
D4

In [ ]:
# en fait si mais avec des noms qu'on n'a pas choisis
x = D4[0]
x['f0']

### dict (1)

In [ ]:
D4 = np.array( d4_data,
       dtype = {'names':['nom', 'matrice'],
                'formats':['S2', '(2,2)int8']})
D4

In [ ]:
# pas de nom pour accéder aux différents éléments
x = D4[2]
print(x)

In [ ]:
y = D4[4]
# mais par contre une fois qu'on a un élément 
# on peut accéder aux deux colonnes par nom
produit = x['matrice'].dot(y['matrice'])
print("{} x {} ->\n{}".format(x['nom'], y['nom'], produit))

### dict (2)

Attention aux offsets: ce **n'est pas** simplement un ordre des champs!

In [ ]:
D4 = np.array( d4_data,
    dtype = {'nom' : ('S2', 0),
             'matrice': ('(2,2)float32', 4)}
)
print(D4)

In [ ]:
x = D4[2]
print(x)

In [ ]:
y = D4[4]
produit = x['matrice'].dot(y['matrice'])
print("{} x {} ->\n{}".format(x['nom'], y['nom'], produit))

## `genfromtxt`

la fonction [`numpy.genfromtxt`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html) permet de construire un tableau numpy à partir d'un fichier texte

In [ ]:
!cat data/D4.txt

In [ ]:
d4_raw = np.genfromtxt("data/D4.txt", 
                       dtype=None)
d4_raw

In [ ]:
d4_raw = np.genfromtxt("data/D4.txt", 
                       dtype=[('nom', 'S2'), 
                              ('matrice', '(2,2)i8')])
d4_raw